In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('books.csv')

In [3]:
print(data.shape)
print(f'Количество пропусков в данных: {data.isna().sum().sum()}')
print(f'Доля пропусков в данных: {round(data.isna().sum().sum() / (data.shape[0]*data.shape[1]), 2)}')

(1200, 29)
Количество пропусков в данных: 14042
Доля пропусков в данных: 0.4


In [4]:
print(f'Всего объектов у каждого признака (если нет пропусков): {data.shape[0]}')
data.isna().sum().sort_values(ascending=False)

Всего объектов у каждого признака (если нет пропусков): 1200


Тип комикса               1199
Типы материала            1199
Назначение                1199
Школьные предметы         1199
Школьные классы           1199
Художник                  1187
Серии комикса             1181
Жанры комикса             1180
Разделы комикса           1179
Переводчик                1077
Издательский бренд         728
cnt_reviews                671
avg_rating                 203
Возрастные ограничения     197
Серия                      193
Тираж                       68
author                      57
Тип обложки                 43
Количество страниц          29
ISBN                        18
old_price                   10
current_price                9
Издательство                 9
Год издания                  3
link                         1
Вес, г                       1
ID товара                    1
title                        1
Размер                       1
dtype: int64

In [5]:
data['cnt_reviews'].fillna('0 оценок', inplace=True)

In [6]:
good_columns = []
bad_columns = []
for column in data.columns:
    all_items = data.shape[0]
    if data.isna().sum().sort_values(ascending=False)[column] / all_items > 0.3:
        bad_columns.append(column)
    else:
        good_columns.append(column)
        
print(f'В признаках {bad_columns} недостаточно объектов, их стоит удалить.')  
print()
print(f'После удаления всех "плохих признаков" остались следующие: {good_columns}')

В признаках ['Издательский бренд', 'Переводчик', 'Жанры комикса', 'Серии комикса', 'Разделы комикса', 'Художник', 'Школьные предметы', 'Назначение', 'Школьные классы', 'Типы материала', 'Тип комикса'] недостаточно объектов, их стоит удалить.

После удаления всех "плохих признаков" остались следующие: ['link', 'current_price', 'old_price', 'avg_rating', 'cnt_reviews', 'title', 'author', 'ID товара', 'Издательство', 'Серия', 'Год издания', 'ISBN', 'Количество страниц', 'Размер', 'Тип обложки', 'Тираж', 'Вес, г', 'Возрастные ограничения']


In [7]:
data = data[good_columns]
data.dropna(inplace=True)
data = data.reset_index(drop=True)

In [8]:
data = data.drop(['ID товара', 'ISBN'], axis=1)

### Обработка конкретных признаков:

In [9]:
# надо применить OHE
print(data['Тип обложки'].value_counts())
print()
print(data['Возрастные ограничения'].value_counts())

Твёрдый переплёт                  416
Мягкий переплёт                   355
Мягкий заламинированный картон     16
Name: Тип обложки, dtype: int64

16+    572
12+    128
18+     42
0+      35
6+      10
Name: Возрастные ограничения, dtype: int64


**OHE**

In [10]:
def OHE_features(data):
    categorical_features = []
    for column in data.columns:
        if len(data[column].value_counts()) <= 10:
            #data[column] = data[column].astype(str)
            categorical_features.append(column)
    
    ohe = OneHotEncoder(sparse=False)
    ohe_data = ohe.fit_transform(data[categorical_features])
    ohe_columns = ohe.get_feature_names_out(categorical_features)
    ohe_data = pd.DataFrame(ohe_data, columns=ohe_columns)
    
    data = data.drop(categorical_features, axis=1)
    data = pd.concat([data, ohe_data], axis=1)
    
    return data

In [11]:
data = OHE_features(data)

In [12]:
data.columns

Index(['link', 'current_price', 'old_price', 'avg_rating', 'cnt_reviews',
       'title', 'author', 'Издательство', 'Серия', 'Год издания',
       'Количество страниц', 'Размер', 'Тираж', 'Вес, г',
       'Тип обложки_Мягкий заламинированный картон',
       'Тип обложки_Мягкий переплёт', 'Тип обложки_Твёрдый переплёт',
       'Возрастные ограничения_0+', 'Возрастные ограничения_12+',
       'Возрастные ограничения_16+', 'Возрастные ограничения_18+',
       'Возрастные ограничения_6+'],
      dtype='object')

**Размер**

In [13]:
def split_sizes(data, size_column):
    data['Толщина'] = data[size_column].dropna().apply(lambda x: float(x.split('x')[0]))
    data['Ширина'] = data[size_column].dropna().apply(lambda x: float(x.split('x')[1]))
    data['Длина'] = data[size_column].dropna().apply(lambda x: float(x.split('x')[2]))
    
    data.drop([size_column], axis=1)
    
    return data

In [14]:
data = split_sizes(data, 'Размер')

**Некоторые количественные признаки**

In [15]:
data['old_price'] = data['old_price'].apply(lambda x: x.split(' ')[0])
data['current_price'] = data['current_price'].apply(lambda x: x.split(' ')[0])
data['cnt_reviews'] = data['cnt_reviews'].str.replace(r'\s+', '')
data['cnt_reviews'] = data['cnt_reviews'].str.replace('оценок', '')

In [16]:
int_features = ['current_price', 'old_price', 'cnt_reviews', 
                'Год издания', 'Количество страниц', 'Тираж']
float_features = ['avg_rating', 'Длина', 'Ширина', 'Толщина', 'Вес, г']

In [17]:
data[int_features] = data[int_features].astype(int)
data[float_features] = data[float_features].astype(float)

# Итог:

In [18]:
data.shape

(787, 25)

In [19]:
data.head()

,link,current_price,old_price,avg_rating,cnt_reviews,title,author,Издательство,Серия,Год издания,...,Тип обложки_Мягкий переплёт,Тип обложки_Твёрдый переплёт,Возрастные ограничения_0+,Возрастные ограничения_12+,Возрастные ограничения_16+,Возрастные ограничения_18+,Возрастные ограничения_6+,Толщина,Ширина,Длина
0,https://www.chitai-gorod.ru/product/kafe-na-kr...,319,436,3.9,1045,Кафе на краю земли,Джон Стрелеки,Эксмо,Кафе на краю земли,2025,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.1,11.5,18.8
1,https://www.chitai-gorod.ru/product/samyy-boga...,487,624,3.8,457,Самый богатый человек в Вавилоне (львы),Джордж Сэмюэль Клейсон,Эксмо,Золотой фонд книг о богатстве,2025,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.8,14.2,20.5
2,https://www.chitai-gorod.ru/product/atomnye-pr...,899,1436,3.9,0,Атомные привычки. Как приобрести хорошие привы...,Джеймс Клир,Питер,Сам себе психолог. Питер,2024,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.6,17.0,24.0
3,https://www.chitai-gorod.ru/product/psihotryuk...,919,1249,3.7,0,"Психотрюки. 69 приемов в общении, которым не у...",Игорь Рызов,Эксмо,Кремлевская школа переговоров,2025,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.5,16.0,21.0
4,https://www.chitai-gorod.ru/product/k-sebe-nez...,855,1096,3.8,6329,"К себе нежно. Книга о том, как ценить и беречь...",Ольга Примаченко,Эксмо,"Книги, которые обнимают. Авторская серия Ольги...",2025,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,14.0,20.5


In [21]:
data.to_csv('data_after_processing.csv', encoding='utf-8')